# Model Evaluation

In this notebook we will choose the 4 models. 

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

from CogniPredictAD.classification import ADNIClassifier

pd.set_option('display.max_rows', 116)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_info_columns', 40) 


train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# Target column
y_train = train['DX']

# All other columns as features
X_train = train.drop(columns=['DX'])

# Target column
y_test = test['DX']

# All other columns as features
X_test = test.drop(columns=['DX'])

classifier = ADNIClassifier()

## Dataset with `CDRSB`, `LDELTOTAL`, and `mPACCdigit` with Classification

**Let's see which of these models performs best on the test set and which one will therefore be chosen. Models ending in 0 are models built without using sampling, while those ending in 1 are models built using Hybrid Sampling.**

In [ ]:
out = classifier.evaluate_models_from_dir(
                models_dir="../results/all_models/1",
                X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                cv_splits=5, cv_repeats=3,
                display_individual_tables=True
            )

## Dataset without `CDRSB`, `LDELTOTAL`, and `mPACCdigit` with Classification

**Let's see which of these models performs best on the test set and which one will therefore be chosen. Models ending in 0 are models built without using sampling, while those ending in 1 are models built using Hybrid Sampling.**

In [ ]:
X_train.drop(columns=['CDRSB', 'LDELTOTAL', 'mPACCdigit'], axis=1, inplace=True)
X_test.drop(columns=['CDRSB', 'LDELTOTAL', 'mPACCdigit'], axis=1, inplace=True)
out = classifier.evaluate_models_from_dir(
                models_dir="../results/all_models/2",
                X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                cv_splits=5, cv_repeats=3,
                display_individual_tables=True
            )

...